In [1]:
import torchtext
from torchtext import data
from torchtext.data import Field
import pandas as pd
import numpy as np
import spacy
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,wordpunct_tokenize
import string
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
nlp = spacy.load('en')

In [3]:
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.RESERVED)

In [4]:
stopwords = set(stopwords.words('english'))

In [5]:
stopwords.add("'s")
stopwords.add("...")

### Creating Dataset using TabularDataset from Torchtext

In [6]:
#TEXT = Field(sequential=True, preprocessing = data.Pipeline(lambda x : text_preprocessing(x)),
#             include_lengths=True)

TEXT = Field(tokenize = 'spacy', stop_words=stopwords, batch_first=True)

#include_lengths - converts TEXT into a tuple of two elements, 
#one element is the numericalised tensor with pads
#other element is the actual length of our sentence

LABEL = data.LabelField(dtype = torch.float)

tweet = data.TabularDataset(path='train_original.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                'target': ('labels', LABEL)
                                   }
                           )

In [7]:
TEXT.build_vocab(tweet,max_size = 15000,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(tweet)

In [8]:
#TEXT.build_vocab(tweet, max_size=10000)

In [9]:
#LABEL.build_vocab(tweet)

In [10]:
LABEL.vocab.freqs

Counter({'1': 3271, '0': 4342})

In [11]:
#TEXT.vocab.freqs.pop('') # remove empty string

In [12]:
train, valid = tweet.split(split_ratio=0.8)

### Convert Dataset into Iterator 

In [13]:
device = torch.device('cuda')

### BucketIterator - 

>**Shuffles the data in iterator**

>**It is recommended that for test data we should use just itertor, since we are evaluating the model.**

In [14]:
train_iter, val_iter, test_iter = data.BucketIterator.splits((train, valid, None),batch_sizes=(128, 128, 0),
                                                            sort_key=lambda x: len(x.text), device=device)

In [15]:
type(train_iter.data()[0].text)

list

In [16]:
type(val_iter.data()[99].text)

list

In [17]:
test = pd.read_csv("data/test.csv")

In [18]:
%%time
test = data.TabularDataset(path='data/test.csv', format='csv',
                            fields={
                                'text': ('text', TEXT),
                                }
                           )

CPU times: user 518 ms, sys: 4.64 ms, total: 523 ms
Wall time: 523 ms


In [19]:
test_iter = data.Iterator(dataset=test, device=device, batch_size=128, sort=False)

## Model Building

In [191]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx, hidden_size):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.conv_0 = nn.Conv1d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[0], embedding_dim))
        
        self.conv_1 = nn.Conv1d(in_channels = 1, 
                                out_channels = n_filters, 
                                kernel_size = (filter_sizes[1], embedding_dim))
        
#         self.conv_2 = nn.Conv1d(in_channels = 1, 
#                                 out_channels = n_filters, 
#                                 kernel_size = (filter_sizes[2], embedding_dim))
        
        self.fc1 = nn.Linear(len(filter_sizes) * n_filters, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_dim)
        
        self.bn1 = nn.BatchNorm1d(num_features=n_filters)
        self.bn2 = nn.BatchNorm1d(num_features=hidden_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        embedded = self.embedding(text)
       
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim] 
        conved_0 = F.relu(self.bn1(self.conv_0(embedded).squeeze(3)))
        conved_1 = F.relu(self.bn1(self.conv_1(embedded).squeeze(3)))
#         conved_2 = self.dropout(F.relu(self.bn1(self.conv_2(embedded).squeeze(3))))
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
#         pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        
        cat = torch.cat((pooled_0, pooled_1), dim = 1)

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        #self.fc2(F.relu(self.fc1(cat)))
            
        x = self.bn2(F.relu(self.dropout(self.fc1(cat))))
            
        return self.fc2(x)

In [209]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3]#4]
OUTPUT_DIM = 1
HIDDEN_SIZE = 256
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX, HIDDEN_SIZE)

In [210]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,602,825 trainable parameters


In [211]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([15002, 100])


In [212]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.4144, -1.7379,  2.0878,  ..., -0.2951,  0.5003, -0.9515],
        [ 0.1726,  1.6152, -0.3193,  ..., -1.0712,  2.1790, -1.2354],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.0750, -1.3663,  0.9371,  ..., -0.2487,  2.2365, -0.4858],
        [ 2.0990,  0.5979,  1.0705,  ...,  0.8962,  0.8123, -0.7778],
        [-0.4421,  0.1331, -0.9618,  ...,  0.1151,  1.3707, -1.5081]])

In [213]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.6610, -0.0730,  0.9238,  ..., -0.2256,  0.8148, -0.4405],
        ...,
        [ 0.0750, -1.3663,  0.9371,  ..., -0.2487,  2.2365, -0.4858],
        [ 2.0990,  0.5979,  1.0705,  ...,  0.8962,  0.8123, -0.7778],
        [-0.4421,  0.1331, -0.9618,  ...,  0.1151,  1.3707, -1.5081]])


In [214]:
TEXT.vocab.stoi[TEXT.unk_token]

0

In [215]:
print(model.embedding.weight.data.shape)

torch.Size([15002, 100])


In [216]:
optimizer = optim.AdamW(model.parameters())

In [217]:
criterion = nn.BCEWithLogitsLoss()

In [218]:
model = model.to(device)
criterion = criterion.to(device)

In [219]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [220]:
model

CNN(
  (embedding): Embedding(15002, 100, padding_idx=1)
  (conv_0): Conv1d(1, 100, kernel_size=(2, 100), stride=(1,))
  (conv_1): Conv1d(1, 100, kernel_size=(3, 100), stride=(1,))
  (fc1): Linear(in_features=200, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1, bias=True)
  (bn1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [221]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.labels)
        
        acc = binary_accuracy(predictions, batch.labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [222]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.labels)
            
            acc = binary_accuracy(predictions, batch.labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [223]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [224]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 1s
	Train Loss: 0.633 | Train Acc: 63.62%
	 Val. Loss: 0.567 |  Val. Acc: 72.91%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.494 | Train Acc: 77.07%
	 Val. Loss: 0.571 |  Val. Acc: 72.06%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.378 | Train Acc: 85.04%
	 Val. Loss: 0.517 |  Val. Acc: 77.10%
Epoch: 04 | Epoch Time: 0m 1s
	Train Loss: 0.265 | Train Acc: 90.31%
	 Val. Loss: 0.545 |  Val. Acc: 76.26%
Epoch: 05 | Epoch Time: 0m 1s
	Train Loss: 0.173 | Train Acc: 94.78%
	 Val. Loss: 0.578 |  Val. Acc: 76.43%
Epoch: 06 | Epoch Time: 0m 1s
	Train Loss: 0.117 | Train Acc: 97.17%
	 Val. Loss: 0.613 |  Val. Acc: 76.24%
Epoch: 07 | Epoch Time: 0m 1s
	Train Loss: 0.103 | Train Acc: 97.53%
	 Val. Loss: 0.752 |  Val. Acc: 73.79%
Epoch: 08 | Epoch Time: 0m 1s
	Train Loss: 0.089 | Train Acc: 97.73%
	 Val. Loss: 0.698 |  Val. Acc: 75.26%
Epoch: 09 | Epoch Time: 0m 1s
	Train Loss: 0.076 | Train Acc: 97.82%
	 Val. Loss: 0.744 |  Val. Acc: 76.15%
Epoch: 10 | Epoch Time: 0m 1

In [208]:
test_preds = []

with torch.no_grad():
    for batch in test_iter:
        predictions = model(batch.text).squeeze(1)
        preds = predictions.data.cpu().numpy()
        #preds = preds.data.numpy()
        # the actual outputs of the model are logits, so we need to pass these values to the sigmoid function
        preds = 1 / (1 + np.exp(-preds))
        test_preds.append(preds)

In [38]:
predictions = np.where(np.concatenate(test_preds, axis=0) > 0.5, 1, 0)

In [39]:
test_data = pd.read_csv('data/test.csv')
test_data["target"] = predictions
submission_file = test_data[['id', 'target']]
submission_file.to_csv("fxt_txt_Submission_file.csv", index=False)